In [ ]:
# with open('./GX/pcNet.py') as pcNet:
#     exec(pcNet.read())

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scanpy as sc
sc.settings.verbosity = 0
from anndata import AnnData
import scipy
from numpy.linalg import svd


import warnings
warnings.filterwarnings("ignore")


In [ ]:
try:
    from sys import path as syspath
    from os import path as ospath
    syspath.append(ospath.join(ospath.expanduser("~"), 'Downloads/Xct/GX/'))
    
    from pcNet import pcNet
    from main import *
    import dNN 
    
except ImportError:
    print('Module not found')

In [ ]:
ada = sc.read_h5ad("LS.h5ad")
ada

In [ ]:
data = csr_matrix.toarray(ada.X)
counts = np.asarray(np.expm1(data), dtype=int)
ada.layers['raw'] = counts
ada.layers['log1p'] = data

In [ ]:
#ada.X = ada.layers['log1p']

In [ ]:
#sc.pp.highly_variable_genes(ada, n_top_genes=3000, flavor='seurat_v3')

In [ ]:
ada.var.head()

In [ ]:
plt.hist(ada.var['vst.mean'], bins=100, density=False)
plt.show()

In [ ]:
plt.hist(ada.var['vst.variance.standardized'], bins=100, density=False)
plt.show()

### HVG: Seurat v3

In [ ]:
HVG_i = np.argsort(np.asarray(ada.var['vst.variance.standardized']))[-3000:]
ada = ada[:, HVG_i].copy()
ada

In [ ]:
ada.var.head()

In [ ]:
plt.hist(ada.var['vst.mean'], bins=100, density=False)
plt.show()

In [ ]:
plt.hist(ada.var['vst.variance.standardized'], bins=100, density=False)
plt.show()

### Xct object

In [ ]:
skin = Xct_metrics(ada)
type(skin)

In [ ]:
skin.chen2016_fit(ada, plot = True, verbose = True)

In [ ]:
obj1 = Xct(ada, 'Inflam. FIB', 'Inflam. DC')
df1 = obj1.fill_metric()
df1

In [ ]:
dir(obj1) #check attributes

In [ ]:
obj1.score(method=0)

In [ ]:
np.random.seed(1)

orig_score0 = np.asarray(obj1.score(method=0))
pmt_scores0 = scores(ada, obj1, n=10)
#pmt_scores0.shape
enriched, pvals, counts = pmt_test(orig_score0, pmt_scores0)
enriched, counts

In [ ]:
df_enriched = df1.iloc[enriched, :]
df_enriched

In [ ]:
ada.obs.head()

In [ ]:
CellA = 'Inflam. FIB' 
CellB = 'Inflam. DC'
ada_A = ada[ada.obs['ident'] == CellA, :].copy()
ada_B = ada[ada.obs['ident'] == CellB, :].copy()
ada_A, ada_B

### w

In [ ]:
# net_A = pcNet(ada_A.X, nComp=5, symmetric=True)
# net_B = pcNet(ada_B.X, nComp=5, symmetric=True)
# net_A.shape, net_B.shape

In [ ]:
# np.savetxt("net_A.csv", net_A, delimiter="\t")
# np.savetxt("net_B.csv", net_B, delimiter="\t")
net_A = np.genfromtxt('./GX/net_A.csv', delimiter="\t")
net_B = np.genfromtxt('./GX/net_B.csv', delimiter="\t")
net_A.shape, net_B.shape

In [ ]:
obj1._metric_A.shape, obj1._metric_B.shape

In [ ]:
# u^2 + var
metric_A_temp = (np.square(obj1._metric_A[0]) + obj1._metric_A[1])[:, None] 
metric_B_temp = (np.square(obj1._metric_B[0]) + obj1._metric_B[1])[None, :] 
print(metric_A_temp.shape, metric_B_temp.shape)

w12 = metric_A_temp@metric_B_temp
w12.shape

In [ ]:
w12

In [ ]:
# w = np.block([[net_A+1, w12],
#             [w12.T, net_B+1]])
# w.shape

#### scale w:

In [ ]:
num = 0.1 * (np.sum(abs(net_A+1)) + np.sum(abs(net_B+1))) / (len(net_A) + len(net_B))
w12_scale = w12*num
num

In [ ]:
w = np.block([[net_A +1, w12_scale],
            [w12_scale.T, net_B +1]])
w.shape

### eigen

In [ ]:
# w_test = np.array([[1, 0, 3, 2], [1, 1, 0, 0], [5, 0, 2, 1], [1, 0, 0, 1]])
# w_test = (w_test + w_test.T)//2
# w_test

In [ ]:
def Manifold_Align(w, d = 3):
    
    def is_symmetric(a, rtol=1e-05, atol=1e-08):
        return np.allclose(a, a.T, rtol=rtol, atol=atol)
    
    if not is_symmetric(w):
        raise ValueError('Input should be a symmetric matrix')
    else:
        L = laplacian(w, normed=False)
        U, s, VT = svd(L, full_matrices=True) 
        #print ('U:', U.shape, 's:', s.shape, 'VT:', VT.shape)
        V = VT.T
        V = V[:, s > 1e-4]
        #print('V:', V.shape)
        V = V[:, :d]
    
        return V


In [ ]:
MA = Manifold_Align(w, d=3)
MA.shape

In [ ]:
def pair_distance(ma, genelist1, genelist2):
    d = {}
    for i, l in enumerate(genelist1):
        for j, r in enumerate(genelist2):
            d[f'{l}_{r}'] = np.linalg.norm(ma[i, :] - ma[len(genelist1) + j, :])
    
    return d

In [ ]:
result_eig = pair_distance(MA, list(ada_A.var_names.astype(str)), list(ada_B.var_names.astype(str)))
len(result_eig)

In [ ]:
plt.hist(result_eig.values(), bins=100)
plt.show()

In [ ]:
#pd.DataFrame.from_dict(result_eig, orient='index').to_csv('distances_eig.csv', header=None)

In [ ]:
df_eig = pd.DataFrame.from_dict(result_eig, orient='index', columns=['dist']).sort_values(by=['dist'])
df_eig['rank'] = np.arange(len(df_eig))
df_eig.head()

In [ ]:
df_eig.loc['CCL19_CCR7']

In [ ]:
np.percentile(np.asarray(df_eig['dist']), 95)

### NN

In [ ]:
x1_np = scipy.sparse.csr_matrix.toarray(ada_A.X.T) #gene by cell
x2_np = scipy.sparse.csr_matrix.toarray(ada_B.X.T)

In [ ]:
n = 3000
lr = 0.001

projections, losses = dNN.train_and_project(x1_np, x2_np, w=w, n=n, lr=lr)

In [ ]:
plt.figure(figsize=(6, 5), dpi=80)

plt.plot(np.arange(len(losses))*100, losses)
#plt.savefig('fig.png', dpi=80)
plt.show()

In [ ]:
df_proj = pd.DataFrame(projections)

df_proj.index = list(ada_A.var_names) + list(ada_B.var_names)
df_proj.head()

In [ ]:
result_nn = pair_distance(projections, list(ada_A.var_names.astype(str)), list(ada_B.var_names.astype(str)))
len(result_nn)

In [ ]:
plt.hist(result_nn.values(), bins=100)
plt.show()

In [ ]:
#pd.DataFrame.from_dict(result_nn, orient='index').to_csv('distances_nn.csv', header=None)

In [ ]:
df_nn = pd.DataFrame.from_dict(result_nn, orient='index', columns=['dist']).sort_values(by=['dist'])
df_nn['rank'] = np.arange(len(df_nn))
df_nn.head()

In [ ]:
df_nn.loc['CCL19_CCR7']

In [ ]:
np.percentile(np.asarray(df_nn['dist']), 5)